In [ ]:
#| eval: true


In [ ]:
import papermill

In [ ]:
tmpdir = '.'

In [ ]:
from IPython.display import display, HTML
from itables import init_notebook_mode, show
import os
init_notebook_mode(all_interactive=False)
# tmpdir = os.environ['TARGETER_TMPDIR']

exec(open(os.path.join(tmpdir, 'targeter.py')).read())
print(tmpdir)
tar_pickle_path = os.path.join(tmpdir, 'targeter.pickle')


import pickle
file = open(tar_pickle_path, 'rb')
tar = pickle.load(file)
file.close()

\newpage

## Introduction

This report contains a snapshot of the specific subpopulation **«`r label(object$target,metadata)` » == `r object$target_reference_level``**. 

This subpopulation is characterized by  performing bivariate analyses, crossing the variable of interest (target) with  potential explanatory variables. Only most relevant (highest discriminative power) may be retained.

Report contains:

*   A summary overview table with selected retained criteria with some available information such as one statistic called information value (IV) that provides an idea of the explanatory power of the criteria.
*   Detailed analyses per criteria with graphics that highlight the reation between one of the candidate criteria and the _target_.

It can happen that one criteria contains some missing values. If such, we will introduce value *"[Missing]"* in the report, eventually with a cross **[x]**. Also, if two few data are available, we would use a minus **[-]** instead of reporting value. 

### Summary table interpretation

Criteria are sorted by decreasing importance order, which one is determined by the IV. 
One modality (value) index is also computed as the proportion of target «`r label(object$target,metadata)`» within the modality, that is the target penetration in the subpopulation selected by the specific modality of a criteria. A value near 1 would indicate a non-discriminative modality: selecting the corresponding subpopulation would result in the overall average target penetration. On the other way, the hightest the index (>1), the more target is over-representated in the subpopulation. Low index (<1) would be interpreted as under-representation. So high index modalities are of interest as they characterize subpopulation with high target penetration.

Note that index will always be computed for segments (modalities): continuous variables are cut into buckets. If a criteria/variable has no segment with any high index, it will not be of interest at all. Also, having one segment with an index  greater than 1 implies that there are counterparts segments with index lower 1.

There is no special rule to say that an index is a __good__ one as its interpretation has to take into account the natural (average) target pourcentage (penetration). Indeed, when the penetration is low, it is easier to observe eventually high index. Think that for a global penetration of 50% of target, maximal index is 2 (and more generally, index is bounded by `1/(target penetration)`)

For this report, we selected a maximum of  `r ntop` potential criteria.

For every criteria, summary table presents its possible value (modality) having the highest index along with the corresponding number of records. Pay attention that you might observe high index dealing with very restrictive sub-populations.

## Population description



## Target description
Target analysis :

In [ ]:
tar.target_stats

## Summary view


In [ ]:
tab_sum = tar.summary()

tab_sum.rename(columns={ 'quality_score':'qs','Max ER - Bin': 'Max bin','Max Event Rate':'MaxBin Rate','Max ER - Count':'MaxBin N'}, inplace = True)

tab_sum.sort_values(by=['qs'], ascending = False)

vars = ['name','n_bins','iv','qs','Max bin','MaxBin Rate','MaxBin N']

tab_sum = tab_sum[vars]

tab_sum = tab_sum.astype({'iv': float, 'qs': float}) # https://dev.to/theolakusibe/solving-attributeerror-float-object-has-no-attribute-rint-4la#:~:text=Solution%20To%20convert%20the%20column%20with%20a%20mixed,by%20using%20the%20pandas.DataFrame.astype%20function%2C%20as%20illustrated%20below

tab_sum['iv'] = tab_sum['iv'].round(3)

tab_sum['qs'] = tab_sum['qs'].round(3)





# add hyperlinks -- # df2$variable <- paste0("<a href='#var-",tolower(df2$variable),"'>",df2$variable,"</a>")

tab_sum['name']= tab_sum['name'].map(lambda cell: "<a href='#var-" + cell.lower() +"'>"+cell+"</a>")

tab_sum['MaxBin Rate'] = tab_sum['MaxBin Rate'].round(3)




# wrap text https://stackoverflow.com/questions/63286431/datatables-js-not-wrapping-cell-text#:~:text=You%20can%20try%20creating%20a%20class%20and%20then,data%20%2B%20%22%3C%2Fdiv%3E%22%3B%20%7D%2C%20targets%3A%202%20%7D%20%5D

#<IDEA> extend itables python package to add advanced search functionality

tab_sum['Max bin'] = tab_sum['Max bin'].map(lambda cell: '<div style="white-space: normal;">'+cell+"<div>")


<!--https://quarto.org/docs/authoring/conditional.html-->

::: {.content-visible when-format="html"}


In [ ]:
# Will only appear in HTML.

show(tab_sum, paging = False, info = False, style = "width:100%;margin:auto")

:::






::: {.content-visible unless-format="html"}


In [ ]:
# Will only appear in HTML.

tab_sum

:::


## Variables


In [ ]:
#| output: asis

import matplotlib.pyplot as pyplot
from adjustText import adjust_text


def label(var, metadata = None):
  if metadata is None:
    return(var)
metadata = None

def format_df(df, index = True):
    """format the dataframe for display"""
    return display(HTML(df.to_html(index=index)))

# loop over variables
for ivar in list(tar.profiles.variable_names):
  # display title+anchor for links

  ivar_label = label(ivar, metadata)
  itxt = "<div id='var-" + ivar.lower() + "'>\n\n## " + ivar_label

  if (ivar_label != ivar):
    itxt = itxt + "(" + ivar + ")"
  itxt = itxt + "\n</div>"

  from IPython.display import display, Markdown
  display(Markdown(f"""{itxt}"""))

#https://stackoverflow.com/questions/74162212/two-columns-layout-in-quarto
# https://quarto.org/docs/authoring/figures.html

  display(Markdown("\n\n::: {<layout='[[45,-10,45],[100]]'}\n\n"))

  tar.plot(ivar)

  tar.quadrant_plot(ivar)

  vars =['Bin','Count', 'Count (%)', 'Event rate','WoE']
  format_df(tar.get_table(ivar)[vars])

  display(Markdown("\n\n:::\n\n"))